# amount spent 예측

In [62]:
import pandas as pd
import numpy as np


In [63]:
x_raw = pd.read_csv('../BigContest/AllMergeJamDonny.csv')
x = x_raw 

In [30]:
from IPython.display import display
pd.options.display.max_columns = None
display(x.head())


,Unnamed: 0,acc_id,playtime,npc_kill,solo_exp,party_exp,quest_exp,boss_monster,death,revive,exp_recovery,fishing,private_shop,game_money_change,enchant_count,level,pledge_cnt,random_attacker_cnt,random_defender_cnt,temp_cnt,same_pledge_cnt,etc_cnt,num_opponent,amount_spent_pay,sell_item_cnt,buy_item_cnt,play_char_cnt,combat_char_cnt,pledge_combat_cnt,random_attacker_cnt_plg,random_defender_cnt_plg,same_pledge_cnt_plg,temp_cnt_plg,etc_cnt_plg,combat_play_time,non_combat_play_time,survival_time,amount_spent
0,0,27835,9.978209,12.747726,0.130439,0.000000,0.022510,1.0,0.245370,0.246819,0.000000,1.199558,0.000000,0.011245,0.0,17.0,0.000000,0.0,0.000000,1.918911,0.000000,2.139619,1.962575,0.873062,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,64.0,0.002559
1,1,12351,82.013292,37.056629,0.532874,0.039412,0.016449,7.0,0.981479,0.987277,0.000000,39.579596,0.000000,1.588348,0.0,17.0,7.124451,0.0,4.732258,5.756733,0.000000,2.364842,12.854867,0.528062,17.0,9.0,28.402294,6.556226,3.782773,2.804439,1.867763,11.316867,11.085888,3.106572,21.074054,0.019254,64.0,0.120154
2,2,125437,40.050302,13.461275,0.176408,0.015579,0.133937,7.0,12.023119,12.094146,0.000000,23.991864,0.000000,-0.060990,0.0,17.0,2.631554,0.0,2.366129,0.000000,0.000000,4.054015,6.770884,1.815358,19.0,57.0,50.380690,29.701510,8.097357,18.106920,3.218910,43.764446,27.291721,35.660301,79.411095,0.000000,55.0,0.182593
3,3,104483,61.668175,48.607348,0.243065,0.001153,0.006926,2.0,1.472219,1.480916,31.830496,2.044150,0.115662,0.087355,0.0,17.0,0.000000,0.0,16.562903,1.918911,0.000000,2.139619,2.747605,0.371990,14.0,12.0,74.043254,8.180246,0.369717,4.109830,3.195066,7.426694,17.930793,1.802598,36.301640,0.000000,64.0,0.016241
4,4,4704,26.002596,17.106278,0.432320,0.545593,0.226839,15.0,0.736109,0.740458,0.000000,9.279142,0.000000,-0.058153,0.0,16.0,1.026948,0.0,3.549193,3.837822,12.398273,3.603569,5.298953,1.404644,26.0,33.0,6.649457,2.526252,0.100853,0.439763,1.502158,1.094111,7.960085,1.635964,7.578028,0.042944,20.0,0.226396


## 전처리

In [31]:
import numpy as np

# 정규 분포를 이용하여 데이터 분포에 타원을 그립니다. 타원에서 벗어날수록 outlier
from sklearn.covariance import EllipticEnvelope
# 랜덤 데이터를 생성하는데 사용됩니다.
from sklearn.datasets import make_blobs
#https://gaussian37.github.io/ml-sklearn-Detecting-Outliers/

### 이상치 제거 


1	One-class SVM	                
Novelty

#### 2	Fitting an elliptic envelope	
Outlier

3	Isolation Forest	            
Outlier

4	Local Outlier Factor	        
Outlier


ref: https://mkjjo.github.io/python/2019/01/10/outlier.html


In [32]:
outlier_detector = EllipticEnvelope(contamination=.1)
outlier_detector.fit(x)
outl = pd.DataFrame(outlier_detector.predict(x))


In [33]:
x= pd.concat([x,outl],axis=1)
x =x.rename(columns={0:'outlier'})
x=x[x['outlier']==1]

In [34]:
# x = x[x['amount_spent_pay']>0] # 과금 유저만.

In [35]:
x.count()

Unnamed: 0                 36000
acc_id                     36000
playtime                   36000
npc_kill                   36000
solo_exp                   36000
party_exp                  36000
quest_exp                  36000
boss_monster               36000
death                      36000
revive                     36000
exp_recovery               36000
fishing                    36000
private_shop               36000
game_money_change          36000
enchant_count              36000
level                      36000
pledge_cnt                 36000
random_attacker_cnt        36000
random_defender_cnt        36000
temp_cnt                   36000
same_pledge_cnt            36000
etc_cnt                    36000
num_opponent               36000
amount_spent_pay           36000
sell_item_cnt              36000
buy_item_cnt               36000
play_char_cnt              36000
combat_char_cnt            36000
pledge_combat_cnt          36000
random_attacker_cnt_plg    36000
random_def

In [36]:
# x = x[(x['amount_spent_pay']>np.percentile(x[['amount_spent_pay']], 25))& (x['amount_spent_pay']<np.percentile(x[['amount_spent_pay']], 75))] # 2-3분위

In [37]:
x[x['amount_spent_pay']<0].count() # payment 동일 유저.

Unnamed: 0                 0
acc_id                     0
playtime                   0
npc_kill                   0
solo_exp                   0
party_exp                  0
quest_exp                  0
boss_monster               0
death                      0
revive                     0
exp_recovery               0
fishing                    0
private_shop               0
game_money_change          0
enchant_count              0
level                      0
pledge_cnt                 0
random_attacker_cnt        0
random_defender_cnt        0
temp_cnt                   0
same_pledge_cnt            0
etc_cnt                    0
num_opponent               0
amount_spent_pay           0
sell_item_cnt              0
buy_item_cnt               0
play_char_cnt              0
combat_char_cnt            0
pledge_combat_cnt          0
random_attacker_cnt_plg    0
random_defender_cnt_plg    0
same_pledge_cnt_plg        0
temp_cnt_plg               0
etc_cnt_plg                0
combat_play_ti

In [38]:
x = x.drop(columns=['survival_time','Unnamed: 0','acc_id'])


In [39]:
len(x.index.values)

36000

In [40]:
x.count()

playtime                   36000
npc_kill                   36000
solo_exp                   36000
party_exp                  36000
quest_exp                  36000
boss_monster               36000
death                      36000
revive                     36000
exp_recovery               36000
fishing                    36000
private_shop               36000
game_money_change          36000
enchant_count              36000
level                      36000
pledge_cnt                 36000
random_attacker_cnt        36000
random_defender_cnt        36000
temp_cnt                   36000
same_pledge_cnt            36000
etc_cnt                    36000
num_opponent               36000
amount_spent_pay           36000
sell_item_cnt              36000
buy_item_cnt               36000
play_char_cnt              36000
combat_char_cnt            36000
pledge_combat_cnt          36000
random_attacker_cnt_plg    36000
random_defender_cnt_plg    36000
same_pledge_cnt_plg        36000
temp_cnt_p

In [41]:
x_data = [ x.iloc[i,:-1].tolist() for i in range(len(x.index.values))]

In [42]:
x_data

[[9.978208852186876,
  12.747726126138609,
  0.1304385436205031,
  0.0,
  0.0225102773123252,
  1.0,
  0.2453697795613059,
  0.24681931234332186,
  0.0,
  1.1995576730576354,
  0.0,
  0.011245308493905749,
  0.0,
  17.0,
  0.0,
  0.0,
  0.0,
  1.9189110569779195,
  0.0,
  2.1396189985508025,
  1.9625750905578725,
  0.8730617954718154,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0025592618245919444],
 [82.01329237820012,
  37.05662940962849,
  0.5328737530715739,
  0.03941204989945244,
  0.01644895823328676,
  7.0,
  0.9814791182452236,
  0.9872772493732874,
  0.0,
  39.579596414776574,
  0.0,
  1.5883484128283167,
  0.0,
  17.0,
  7.124450931091294,
  0.0,
  4.732257939520743,
  5.756733170933758,
  0.0,
  2.3648420510298345,
  12.854866843154063,
  0.5280615698418238,
  17.0,
  9.0,
  28.402294328847017,
  6.5562263486749774,
  3.7827733970406,
  2.8044387805854822,
  1.8677627359825,
  11.316866840938628,
  11.085888360546141,
  3.106571987

In [43]:
X = x_data

In [44]:
y_data = [[i] for i in x['amount_spent'].tolist() ]

In [45]:
y= y_data

## test, train 분할

In [46]:
from sklearn.model_selection import train_test_split

x_data, X_test, y_data, y_test = train_test_split(X, y, test_size = 1/3, random_state = 70) #42

## Scaler

1	StandardScaler	
기본 스케일. 평균과 표준편차 사용


#### 2	MinMaxScaler	
최대/최소값이 각각 1, 0이 되도록 스케일링


3	MaxAbsScaler	
최대절대값과 0이 각각 1, 0이 되도록 스케일링


4	RobustScaler	
중앙값(median)과 IQR(interquartile range) 사용. 아웃라이어의 영향을 최소화


#### MinMaxScaler

In [47]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaler.fit(x_data)
data = scaler.transform(x_data)
x_data = data

In [48]:
scaler.fit(X_test)
data = scaler.transform(X_test)
X_test = data

#### RobustScaler

https://mkjjo.github.io/python/2019/01/10/scaler.html

In [49]:
# from sklearn.preprocessing import RobustScaler
# robustScaler = RobustScaler()
# print(robustScaler.fit(x_data))
# train_data_robustScaled = robustScaler.transform(x_data)
# x_data = train_data_robustScaled

In [50]:
# print(robustScaler.fit(X_test))
# train_data_robustScaled = robustScaler.transform(X_test)
# X_test = train_data_robustScaled

In [51]:
x_data = np.array(x_data)

In [52]:
x_data.shape

(24000, 35)

In [53]:
len(y_data)

24000

## Model (multi_variable_linear_regression)

In [54]:
# multi_variable_linear_regression.py
# 4

import tensorflow as tf


X = tf.placeholder(tf.float32,shape=[None,x_data.shape[1]])
Y = tf.placeholder(tf.float32,shape=[None,1])

W = tf.Variable(tf.random_normal([x_data.shape[1],1]), name='weight')
b = tf.Variable(tf.random_normal([1]), name='bias')

hypothesis = tf.matmul(X,W) + b  #

cost = tf.reduce_mean(tf.square(hypothesis - Y))

optimizer = tf.train.AdamOptimizer(learning_rate=3e-1)
train = optimizer.minimize(cost)



# RMSE( root-mean-square error ) : 평균 제곱 근 오차
# 평균 제곱근편차(RMSD) 또는 평균 제곱근오차(RMSE)는 추정값 또는 모델이 예측한 값과
# 실제 환경에서 관찰되는 값의 차이를 다룰 때 흔히 사용하는 측도이다.
# 정밀도를 표현하는데 적합하다.

targets = tf.placeholder(tf.float32,[None,1])  # 답(Y)
predictions = tf.placeholder(tf.float32,[None,1]) # 예측 값
rmse = tf.sqrt(tf.reduce_mean(tf.square(targets-predictions)))

sess = tf.Session()
sess.run(tf.global_variables_initializer())

# start training
for step in range(10001):
    cost_val, W_val, b_val, _ = \
        sess.run([cost, W, b, train],
                 feed_dict={X:x_data, Y:y_data})
    if step % 1000 == 0:
        print(step, cost_val, W_val, b_val)

# Test step
test_predict = sess.run(hypothesis,feed_dict={X:X_test})
rmse_val = sess.run(rmse,feed_dict={targets:y_test,predictions:test_predict})
print('RMSE: {}'.format(rmse_val))


total_error = tf.reduce_sum(tf.square(tf.subtract(y_test, tf.reduce_mean(y_test))))
unexplained_error = tf.reduce_sum(tf.square(tf.subtract(y_test, test_predict)))
R_squared = tf.subtract(1.0,tf.divide( unexplained_error,total_error))

print('R_sqr: {}'.format(sess.run(R_squared)))

prd = sess.run(hypothesis, feed_dict = {X:X_test})


0 0.86526173 [[-0.02312111]
 [-0.7866982 ]
 [-2.1088486 ]
 [-0.77091795]
 [-0.5610714 ]
 [ 1.0802016 ]
 [-0.03064127]
 [ 0.3415163 ]
 [ 0.9778623 ]
 [-0.99304265]
 [-1.3447199 ]
 [ 0.25738516]
 [ 0.7391002 ]
 [ 0.2731961 ]
 [ 0.93309176]
 [ 0.9575495 ]
 [ 0.543618  ]
 [-0.03574296]
 [-0.21187465]
 [ 1.0747246 ]
 [ 0.7396954 ]
 [-1.00601   ]
 [-0.91544914]
 [-0.47127235]
 [ 0.5935703 ]
 [-0.11383691]
 [ 0.29716516]
 [ 0.6281483 ]
 [-1.0470765 ]
 [-0.08527038]
 [-0.11290272]
 [ 0.20496094]
 [-0.8793119 ]
 [ 1.2058775 ]
 [-0.8495509 ]] [-0.26446334]
1000 1.3035261e-06 [[ 1.5444687e-04]
 [-1.9376681e-04]
 [-2.2277196e-05]
 [-2.3188142e-05]
 [-2.5487450e-04]
 [-2.2563081e-05]
 [-2.3961617e-03]
 [ 2.2497920e-03]
 [ 7.7023818e-05]
 [-4.7294361e-05]
 [ 2.7183504e-04]
 [ 1.2538894e-01]
 [ 1.3196951e-04]
 [ 3.6992616e-04]
 [ 6.0247588e-05]
 [ 1.6541047e-04]
 [-1.7914760e-04]
 [-7.4883114e-04]
 [-6.5310294e-04]
 [-2.5409812e-04]
 [ 2.6533435e-04]
 [-8.9537891e-05]
 [-7.2716409e-04]
 [ 2.4274163e-

In [46]:
col = list(x.columns)

col.pop(-1)
col

['playtime',
 'npc_kill',
 'solo_exp',
 'party_exp',
 'quest_exp',
 'boss_monster',
 'death',
 'revive',
 'exp_recovery',
 'fishing',
 'private_shop',
 'game_money_change',
 'enchant_count',
 'level',
 'pledge_cnt',
 'random_attacker_cnt',
 'random_defender_cnt',
 'temp_cnt',
 'same_pledge_cnt',
 'etc_cnt',
 'num_opponent',
 'amount_spent_pay',
 'sell_item_cnt',
 'buy_item_cnt',
 'play_char_cnt',
 'combat_char_cnt',
 'pledge_combat_cnt',
 'random_attacker_cnt_plg',
 'random_defender_cnt_plg',
 'same_pledge_cnt_plg',
 'temp_cnt_plg',
 'etc_cnt_plg',
 'combat_play_time',
 'non_combat_play_time']

In [205]:
w = pd.DataFrame(W_val)
w = w.rename(columns = {0: 'w'})
c = pd.DataFrame(col)
c = c.rename(columns = {0: 'colName'})

In [206]:
result = pd.concat([c,w],axis=1)
result

,colName,w
0,random_attacker_cnt,0.334212
1,etc_cnt,0.010443
2,same_pledge_cnt_plg,0.307046


In [164]:
reTry= list(result[result['w']<0].colName)
reTry # 제외할 컬럼들

['sell_item_cnt', 'combat_play_time']

In [20]:
prd

array([[0.40206873],
       [0.43975586],
       [0.21693742],
       ...,
       [0.5914906 ],
       [0.47674537],
       [0.42076862]], dtype=float32)

In [60]:
pd.options.display.float_format = '{:.7f}'.format
#https://financedata.github.io/posts/pandas-display-format.html
#pandas 실수방식으로 표현

In [61]:
from IPython.display import display
pd.options.display.max_rows = None

display(pd.concat([pd.DataFrame(prd),pd.DataFrame(y_test)],axis=1))




,0,0
0,0.0322988,0.0360922
1,0.1996944,0.2231479
2,0.0000000,0.0000000
3,0.0446898,0.0499384
4,0.0000000,0.0000000
5,0.0963971,0.1077187
6,0.1337463,0.1494543
7,0.0220219,0.0246083
8,0.0009102,0.0010171
9,0.0000000,0.0000000


In [ ]:
# 0.001 (1e-3)-> 
# 0.003 (3e-3)-> 
# 0.01 (1e-2)-> 
# 0.03 (3e-2)-> 
# 0.1 (1e-1)-> 
# 0.3 (3e-1)-> 
# 1

In [71]:
payment = x_raw

In [77]:
payment[payment['amount_spent_pay']==0].count()

Unnamed: 0                 16274
acc_id                     16274
playtime                   16274
npc_kill                   16274
solo_exp                   16274
party_exp                  16274
quest_exp                  16274
boss_monster               16274
death                      16274
revive                     16274
exp_recovery               16274
fishing                    16274
private_shop               16274
game_money_change          16274
enchant_count              16274
level                      16274
pledge_cnt                 16274
random_attacker_cnt        16274
random_defender_cnt        16274
temp_cnt                   16274
same_pledge_cnt            16274
etc_cnt                    16274
num_opponent               16274
amount_spent_pay           16274
sell_item_cnt              16274
buy_item_cnt               16274
play_char_cnt              16274
combat_char_cnt            16274
pledge_combat_cnt          16274
random_attacker_cnt_plg    16274
random_def

In [122]:
payment.describe()[['amount_spent_pay']]

,amount_spent_pay
count,40000.000000
mean,1.109743
std,1.858979
min,0.000000
25%,0.000000
50%,0.348521
75%,1.310766
max,12.556131


In [156]:
x.describe()[['amount_spent_pay']] # 0원 제외

,amount_spent_pay
count,23726.000000
mean,1.870931
std,2.098117
min,0.011735
25%,0.371990
50%,1.056123
75%,2.461911
max,12.556131


In [150]:
from scipy.stats import describe
describe(x)

DescribeResult(nobs=23726, minmax=(array([ 2.37012087e-03,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00, -8.90657004e+01,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  1.17347016e-02,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00]), array([  92.94903006,  334.39257588,  238.79182324,  300.14545253,
       1146.30416302,   99.        ,  614.6512978 ,  617.7887388 ,
        286.47446016,   84.31456123,   92.5300717 ,  200.16364807,
        565.34453953,   17.        ,  637.15697651,  826.55677045,
        154.98144752,  169.34390078, 

In [153]:
np.percentile(x[['amount_spent_pay']], 25) # 1사분위수

0.3719900391996848

In [154]:
np.percentile(x[['amount_spent_pay']], 50)  # 2사분위수

1.0561231396836477

In [157]:
np.percentile(x[['amount_spent_pay']], 75)  # 3사분위수

2.4619110488642235

In [152]:
np.percentile(x[['amount_spent_pay']], 100)  # 최대

12.556130660683367